# Exercise Sheet 1: Python Basics

This first  exercise sheet tests the basic functionalities of the Python programming language in the context of a simple prediction task. We consider the problem of predicting health risk of subjects from personal data and habits. We first use for this task a decision tree

![](tree.png)

adapted from the webpage http://www.refactorthis.net/post/2013/04/10/Machine-Learning-tutorial-How-to-create-a-decision-tree-in-RapidMiner-using-the-Titanic-passenger-data-set.aspx. For this exercise sheet, you are required to use only pure Python, and to not import any module, including numpy. In exercise sheet 2, the nearest neighbor part of this exercise sheet will be revisited with numpy.

## Classifying a single instance (15 P)

* Create a function that takes as input a tuple containing values for attributes (smoker,age,diet), and computes the output of the decision tree.
* Test your function on the tuple `('yes',31,'good')`,

In [1]:
#Classifying a single instance (15 P)
def healthClassifier (tpl):
    decision = ""
    if (len(tpl) != 3):
        return "wrong type of tuple"

    if tpl[0] == "yes":
        if int(tpl[1]) <= 29.5:
            decision = "less"
        elif int(tpl[1]) > 29.5:
            decision = "more"
    elif tpl[0] == "no":
        if tpl[2] == "good":
            decision = "less"
        elif tpl[2] == "poor":
            decision = "more"
        else:
            return "invalid answer for diet"
    else:
        return "invalid answer for if smoker"
    return tpl, decision


healthClassifier(("yes",31,"good"))

(('yes', 31, 'good'), 'more')

## Reading a dataset from a text file (10 P)

The file `health-test.txt` contains several fictious records of personal data and habits.

* Read the file automatically using the methods introduced during the lecture.
* Represent the dataset as a list of tuples.

In [2]:
#Reading a dataset from a text file (10 P)
def readTestSet():
    f = open("health-test.txt", "r")
    healthlist = []
    for line in f:
        newEntry = line[:-1].split(',')
        newEntry[1] = int(newEntry[1])
        healthlist = healthlist + [tuple(newEntry)]
    return healthlist


readTestSet()

[('yes', 21, 'poor'),
 ('no', 50, 'good'),
 ('no', 23, 'good'),
 ('yes', 45, 'poor'),
 ('yes', 51, 'good'),
 ('no', 60, 'good'),
 ('no', 15, 'poor'),
 ('no', 18, 'good')]

## Applying the decision tree to the dataset (15 P)

* Apply the decision tree to all points in the dataset, and compute the percentage of them that are classified as "more".

In [3]:
#Applying the decision tree to the dataset (15 P)
def moreRiskPercentage(healthlist):
    evaluatedList = map(healthClassifier, healthlist)
    numMr = 0.0
    numLr = 0.0
    for entry in evaluatedList:
        if entry[1] == "more":
            numMr += 1.0
        elif entry[1] == "less":
            numLr += 1.0
    if numLr+numMr <= 0:
        return "tragic error!"
    return numMr / (numLr+numMr)

moreRiskPercentage(readTestSet())

0.375

## Learning from examples (10 P)

Suppose that instead of relying on a fixed decision tree, we would like to use a data-driven approach where data points are classified based on a set of training observations manually labeled by experts. Such labeled dataset is available in the file `health-train.txt`. The first three columns have the same meaning than for `health-test.txt`, and the last column corresponds to the labels.

* Write a procedure that reads this file and converts it into a list of pairs. The first element of each pair is a triplet of attributes, and the second element is the label.

In [4]:
#Learning from examples (10 P)
def readTrainingSet():
    f = open("health-train.txt", "r")
    healthlist = []
    for line in f:
        newEntry = line[:-1].split(',')
        newEntry[1] = int(newEntry[1])
        healthlist = healthlist + [tuple([tuple(newEntry[:3]), newEntry[3]])]
    return healthlist

readTrainingSet()

[(('yes', 54, 'good'), 'less'),
 (('no', 55, 'good'), 'less'),
 (('no', 26, 'good'), 'less'),
 (('yes', 40, 'good'), 'more'),
 (('yes', 25, 'poor'), 'less'),
 (('no', 13, 'poor'), 'more'),
 (('no', 15, 'good'), 'less'),
 (('no', 50, 'poor'), 'more'),
 (('yes', 33, 'good'), 'more'),
 (('no', 35, 'good'), 'less'),
 (('no', 41, 'good'), 'less'),
 (('yes', 30, 'poor'), 'more'),
 (('no', 39, 'poor'), 'more'),
 (('no', 20, 'good'), 'less'),
 (('yes', 18, 'poor'), 'less'),
 (('yes', 55, 'good'), 'more')]

## Nearest neighbor classifier (25 P)

We consider the nearest neighbor algorithm that classifies test points following the label of the nearest neighbor in the training data. For this, we need to define a distance function between data points. We define it to be

`d(a,b) = (a[0]!=b[0])+((a[1]-b[1])/50.0)**2+(a[2]!=b[2])`

where `a` and `b` are two tuples corrsponding to the attributes of two data points.

* Write a function that retrieves for a test point the nearest neighbor in the training set, and classifies the test point accordingly.
* Test your function on the tuple `('yes',31,'good')`

In [5]:
def d(a,b):
    return (a[0]!=b[0])+((a[1]-b[1])/50.0)**2+(a[2]!=b[2])

def nearestNeighborClassifier(tpl):
    trainingSet = readTrainingSet()
    closest = ["", "none"]
    for entry in trainingSet:
        currentDistance = d(tpl, entry[0])
        if (currentDistance < closest[1]):
            closest = [entry, currentDistance]

    return tuple([tpl, closest[0][1]])


nearestNeighborClassifier(("yes",31,"good"))

(('yes', 31, 'good'), 'more')

* Apply both the decision tree and nearest neighbor classifiers on the test set, and find the data point(s) for which the two classifiers disagree, and with which probability it happens.

In [6]:
def conflictProbability():
    testSet = readTestSet()
    clist = []
    for entry in testSet:
        classified = healthClassifier(entry)
        nnClassified = nearestNeighborClassifier(entry)
        #print "comparing: ", classified, nnClassified
        if classified[1] != nnClassified[1]:
            clist.append(classified[0])

    return tuple([clist, float(len(clist))/float(len(testSet))])

print conflictProbability()

([('yes', 51, 'good')], 0.125)


One problem of simple nearest neighbors is that one needs to compare the point to predict to all data points in the training set. This can be slow for datasets of thousands of points or more. Alternatively, some classifiers train a model first, and then use it to classify the data.

## Nearest mean classifier (25 P)

We consider one such trainable model, which operates in two steps:

(1) Compute the average point for each class, (2) classify new points to be of the class whose average point is nearest to the point to predict.

For this classifier, we convert the attributes smoker and diet to real values (for smoker: yes=1.0 and no=0.0, and for diet: good=0.0 and poor=1.0), and use the modified distance function:

`d(a,b) = (a[0]-b[0])**2+((a[1]-b[1])/50.0)**2+(a[2]-b[2])**2`

We adopt an object-oriented approach for building this classifier.

* Implement the methods `train` and `predict` of the class `NearestMeanClassifier`.

In [7]:
class NearestMeanClassifier:
    
    def dAvg(a,b):
        return (a[0]-b[0])**2+((a[1]-b[1])/50.0)**2+(a[2]-b[2])**2
    
    # Training method that takes as input a dataset
    # and produces two internal vectors corresponding
    # to the mean of each class.
    def train(self,dataset):
        ### Replace by your own code
        pass
        ###
    
    # Prediction method that takes as input a new data
    # point and predicts it to belong to the class with
    # nearest mean.
    def predict(self,x):
        ### Replace by your own code
        pass
        ###

* Build an object of class `NearestMeanClassifier`, train it on the training data, and print the mean vector for each class.

In [8]:
nmc = NearestMeanClassifier()
for entry in readTestSet():
    print nmc.avg


AttributeError: NearestMeanClassifier instance has no attribute 'avg'

* Predict the test data using the nearest mean classifier and print all test examples for which all three classifiers (decision tree, nearest neighbor and nearest mean) agree.

In [9]:
### Replace by your own code
import solutions
solutions.exercise6b()
###

ImportError: No module named solutions